## Processing acc data (2022 dataset)

In [1]:
import math
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/Participant19/acc.csv"
data = pd.read_csv(path)
data.head()

,id,p_id,itr_id,prj_id,ppt_id,ppt_name,rating,hand,clt_id,x,y,z,timestamp
0,261349,19,152,622,12,smoothness,2,0,25,0.246597,-0.457504,0.711624,2022-02-13 12:00:21.557
1,261351,19,152,622,12,smoothness,2,0,25,0.292984,-0.449203,0.783890,2022-02-13 12:00:21.578
2,261934,19,152,622,12,smoothness,2,0,25,0.298355,-0.512192,0.779984,2022-02-13 12:00:27.599
3,261940,19,152,622,12,smoothness,2,0,25,0.304703,-0.507309,0.790726,2022-02-13 12:00:27.655
4,262034,19,152,622,12,smoothness,2,0,25,0.305191,-0.472641,0.733597,2022-02-13 12:00:28.678


### Step 1 - Remove the ppt_name column

In [3]:
for participant in range(19,26):
    print("Participant: ", participant)
    if participant != 20:

        path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/acc.csv"
        data = pd.read_csv(path)
        
        data.drop(['ppt_name'], axis=1, inplace=True)

        write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/acc_prop.csv"
        data.to_csv(write_path, index = None)

Participant:  19
Participant:  20
Participant:  21
Participant:  22
Participant:  23
Participant:  24
Participant:  25


In [4]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(24)+"/acc_prop.csv"
data = pd.read_csv(path)
data.head()

,id,p_id,itr_id,prj_id,ppt_id,rating,hand,clt_id,x,y,z,timestamp
0,462559,24,296,622,12,5,0,50,0.484985,-0.702011,0.355591,2022-02-14 15:06:19.535
1,462056,24,296,622,12,5,0,50,0.493286,-0.779648,0.305786,2022-02-14 15:06:14.285
2,461970,24,296,622,12,5,0,50,0.495728,-0.791367,0.264282,2022-02-14 15:06:13.442
3,462058,24,296,622,12,5,0,50,0.496704,-0.790878,0.289185,2022-02-14 15:06:14.305
4,462557,24,296,622,12,5,0,50,0.497681,-0.710312,0.295044,2022-02-14 15:06:19.520


### Step 2 - Sort the files in temporal order

In [5]:
for participant in range(19,26):
    if participant != 20:

        path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/acc_prop.csv"
        data = pd.read_csv(path)
        print("Participant: ", participant)

        data['timestamp'] = pd.to_datetime(data['timestamp'])

        data.sort_values(by=['timestamp'], inplace=True)


        write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/acc_prop_sorted.csv"
        data.to_csv(write_path,index=False)
        
        

Participant:  19
Participant:  21
Participant:  22
Participant:  23
Participant:  24
Participant:  25


### Step 3 - Compute accelleration, velocity and jerkiness for x, y and z directions from the (linear) accelerometer data we have

In [6]:
labels = ['id','p_id','itr_id','prj_id','ppt_id','rating','hand','clt_id','Ax','Ay','Az','Vx','Vy','Vz','Jx','Jy','Jz','timestamp']

for participant in range(19,26):
    print("Participant: ", participant)
    if participant != 20:

        path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/acc_prop_sorted.csv"
        data = pd.read_csv(path)

        #path = "/Users/niharawarawita/Desktop/MSc Project/Data/Lili_Experiment/"+"participant0"+str(participant)+"/acc_prop.csv"

        data = pd.read_csv(path)


        data['timestamp']=pd.to_datetime(data['timestamp'])
        #data['timestamp']= data['timestamp'].to_timestamp()

        #data['x'] = data['x']/65536.0
        #data['y'] = data['y']/65536.0
        #data['z'] = data['z']/65536.0

        hand0 = data[data['hand'] == 0]
        hand1 = data[data['hand'] == 1]


        speed0x = [0]*len(hand0)
        speed1x = [0]*len(hand1)

        speed0y = [0]*len(hand0)
        speed1y = [0]*len(hand1)

        speed0z = [0]*len(hand0)
        speed1z = [0]*len(hand1)

        jerk0x = [0]*len(hand0)
        jerk1x = [0]*len(hand1)

        jerk0y = [0]*len(hand0)
        jerk1y = [0]*len(hand1)

        jerk0z = [0]*len(hand0)
        jerk1z = [0]*len(hand1)

        for i in range(1,len(hand0)):
            if hand0.iloc[i].at['itr_id'] != hand0.iloc[i-1].at['itr_id']:
                speed0x[i] = 0
                speed0y[i] = 0
                speed0z[i] = 0
                jerk0x[i] = 0
                jerk0y[i] = 0
                jerk0z[i] = 0
            else:    
                time1 = hand0.iloc[i-1]['timestamp']
                time2 = hand0.iloc[i]['timestamp']
                vel_x = (hand0['x'].iloc[i] - hand0['x'].iloc[i-1])*(time2-time1).total_seconds()
                vel_y = (hand0['y'].iloc[i] - hand0['y'].iloc[i-1])*(time2-time1).total_seconds()
                vel_z = (hand0['z'].iloc[i] - hand0['z'].iloc[i-1])*(time2-time1).total_seconds()
                jerk_x = (hand0['x'].iloc[i] - hand0['x'].iloc[i-1])/(time2-time1).total_seconds()
                jerk_y = (hand0['y'].iloc[i] - hand0['y'].iloc[i-1])/(time2-time1).total_seconds()
                jerk_z = (hand0['z'].iloc[i] - hand0['z'].iloc[i-1])/(time2-time1).total_seconds()
                speed0x[i]=vel_x
                speed0y[i]=vel_y
                speed0z[i]=vel_z
                jerk0x[i]=jerk_x
                jerk0y[i]=jerk_y
                jerk0z[i]=jerk_z



        for i in range(1,len(hand1)):
            if hand1.iloc[i].at['itr_id'] != hand1.iloc[i-1].at['itr_id']:
                speed1x[i] = 0
                speed1y[i] = 0
                speed1z[i] = 0
                jerk1x[i] = 0
                jerk1y[i] = 0
                jerk1z[i] = 0
            else:
                time1 = hand1.iloc[i-1]['timestamp']
                time2 = hand1.iloc[i]['timestamp']
                vel_x = (hand1['x'].iloc[i] - hand1['x'].iloc[i-1])*(time2-time1).total_seconds()
                vel_y = (hand1['y'].iloc[i] - hand1['y'].iloc[i-1])*(time2-time1).total_seconds()
                vel_z = (hand1['z'].iloc[i] - hand1['z'].iloc[i-1])*(time2-time1).total_seconds()
                jerk_x = (hand1['x'].iloc[i] - hand1['x'].iloc[i-1])/(time2-time1).total_seconds()
                jerk_y = (hand1['y'].iloc[i] - hand1['y'].iloc[i-1])/(time2-time1).total_seconds()
                jerk_z = (hand1['z'].iloc[i] - hand1['z'].iloc[i-1])/(time2-time1).total_seconds()
                speed1x[i]=vel_x
                speed1y[i]=vel_y
                speed1z[i]=vel_z
                jerk1x[i]=jerk_x
                jerk1y[i]=jerk_y
                jerk1z[i]=jerk_z

        tot0 = pd.DataFrame(hand0, columns=labels)
        tot0['Ax'] = hand0['x']
        tot0['Ay'] = hand0['y']
        tot0['Az'] = hand0['z']
        tot0['Vx'] = speed0x
        tot0['Vy'] = speed0y
        tot0['Vz'] = speed0z
        tot0['Jx'] = jerk0x
        tot0['Jy'] = jerk0y
        tot0['Jz'] = jerk0z

        tot1 = pd.DataFrame(hand1, columns=labels)
        tot1['Ax'] = hand1['x']
        tot1['Ay'] = hand1['y']
        tot1['Az'] = hand1['z']
        tot1['Vx'] = speed1x
        tot1['Vy'] = speed1y
        tot1['Vz'] = speed1z
        tot1['Jx'] = jerk1x
        tot1['Jy'] = jerk1y
        tot1['Jz'] = jerk1z
        #tot = pd.DataFrame([hand0['id'],hand0['p_id'],hand0['e_id'],hand0['itr_id'],hand0['itr_type'],hand0['clt_id'],hand0['hand'],
        #    hand0['x'],hand0['y'],hand0['z'],speed0x,speed0y,speed0z,hand0['timestamp']],columns=labels)

        tot3 = pd.concat([tot0,tot1])
        write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(participant)+"/acc_speeds.csv"
        tot3.to_csv(write_path, index = None)


Participant:  19
Participant:  20
Participant:  21
Participant:  22
Participant:  23
Participant:  24
Participant:  25


In [7]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(25)+"/acc_speeds.csv"
#print(path)
data = pd.read_csv(path)
data.head()



,id,p_id,itr_id,prj_id,ppt_id,rating,hand,clt_id,Ax,Ay,Az,Vx,Vy,Vz,Jx,Jy,Jz,timestamp
0,493631,25,319,622,13,4,0,59,0.799927,-0.455429,0.249634,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-02-15 11:36:03.209
1,493633,25,319,622,13,4,0,59,0.833618,-0.395859,0.410767,0.001482,0.002621,0.007090,0.765714,1.353871,3.662109,2022-02-15 11:36:03.253
2,493636,25,319,622,13,4,0,59,0.857056,-0.295761,0.557739,0.000797,0.003403,0.004997,0.689338,2.944049,4.322725,2022-02-15 11:36:03.287
3,493637,25,319,622,13,4,0,59,0.833618,-0.362656,0.424927,-0.000328,-0.000937,-0.001859,-1.674107,-4.778181,-9.486607,2022-02-15 11:36:03.301
4,493639,25,319,622,13,4,0,59,0.808716,-0.422226,0.295532,-0.000722,-0.001728,-0.003752,-0.858702,-2.054149,-4.461880,2022-02-15 11:36:03.330


### Step 4 - Combine the datasets

In [8]:
path1 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(19)+"/acc_speeds.csv"
data1 = pd.read_csv(path1)

path2 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(21)+"/acc_speeds.csv"
data2 = pd.read_csv(path2)

path3 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(22)+"/acc_speeds.csv"
data3 = pd.read_csv(path3)

path4 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(23)+"/acc_speeds.csv"
data4 = pd.read_csv(path4)

path5 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(24)+"/acc_speeds.csv"
data5 = pd.read_csv(path5)

path6 = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/"+"Participant"+str(25)+"/acc_speeds.csv"
data6 = pd.read_csv(path6)

In [9]:
acc_combined_1 = pd.concat([data1, data2, data3, data4, data5, data6], axis=0)

In [10]:
write_path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/acc_combined_2022.csv"
acc_combined_1.to_csv(write_path, index = None)



In [11]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data and code/2022_Experiment/acc_combined_2022.csv"
data = pd.read_csv(path)


data = pd.read_csv(path)
len(data.itr_id.unique())


216

In [12]:
len(data.itr_id.unique())

216